In [1]:
import pandas as pd
import numpy as np
import string
import spacy
from nltk.corpus import stopwords
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
!pip install textblob

In [2]:
!curl -O https://proai-datasets.s3.eu-west-3.amazonaws.com/fake_news.zip
!unzip fake_news.zip


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 40.9M  100 40.9M    0     0   510k      0  0:01:22  0:01:22 --:--:-- 6649k


In [3]:
# Loading data
df_true = pd.read_csv("True.csv")
df_true

title  \
0      As U.S. budget fight looms, Republicans flip t...   
1      U.S. military to accept transgender recruits o...   
2      Senior U.S. Republican senator: 'Let Mr. Muell...   
3      FBI Russia probe helped by Australian diplomat...   
4      Trump wants Postal Service to charge 'much mor...   
...                                                  ...   
21412  'Fully committed' NATO backs new U.S. approach...   
21413  LexisNexis withdrew two products from Chinese ...   
21414  Minsk cultural hub becomes haven from authorities   
21415  Vatican upbeat on possibility of Pope Francis ...   
21416  Indonesia to buy $1.14 billion worth of Russia...   

                                                    text       subject  \
0      WASHINGTON (Reuters) - The head of a conservat...  politicsNews   
1      WASHINGTON (Reuters) - Transgender people will...  politicsNews   
2      WASHINGTON (Reuters) - The special counsel inv...  politicsNews   
3      WASHINGTON (Reuters) - Trump campaign adviser ...  politicsNews   
4      SEATTLE/WASHINGTON (Reuters) - President Donal...  politicsNews   
...                                                  ...           ...   
21412  BRUSSELS (Reuters) - NATO allies on Tuesday we...     worldnews   
21413  LONDON (Reuters) - LexisNexis, a provider of l...     worldnews   
21414  MINSK (Reuters) - In the shadow of disused Sov...     worldnews   
21415  MOSCOW (Reuters) - Vatican Secretary of State ...     worldnews   
21416  JAKARTA (Reuters) - Indonesia will buy 11 Sukh...     worldnews   

                     date  
0      December 31, 2017   
1      December 29, 2017   
2      December 31, 2017   
3      December 30, 2017   
4      December 29, 2017   
...                   ...  
21412    August 22, 2017   
21413    August 22, 2017   
21414    August 22, 2017   
21415    August 22, 2017   
21416    August 22, 2017   

[21417 rows x 4 columns]

In [4]:
df_fake = pd.read_csv("Fake.csv")
df_fake

title  \
0       Donald Trump Sends Out Embarrassing New Year’...   
1       Drunk Bragging Trump Staffer Started Russian ...   
2       Sheriff David Clarke Becomes An Internet Joke...   
3       Trump Is So Obsessed He Even Has Obama’s Name...   
4       Pope Francis Just Called Out Donald Trump Dur...   
...                                                  ...   
23476  McPain: John McCain Furious That Iran Treated ...   
23477  JUSTICE? Yahoo Settles E-mail Privacy Class-ac...   
23478  Sunnistan: US and Allied ‘Safe Zone’ Plan to T...   
23479  How to Blow $700 Million: Al Jazeera America F...   
23480  10 U.S. Navy Sailors Held by Iranian Military ...   

                                                    text      subject  \
0      Donald Trump just couldn t wish all Americans ...         News   
1      House Intelligence Committee Chairman Devin Nu...         News   
2      On Friday, it was revealed that former Milwauk...         News   
3      On Christmas day, Donald Trump announced that ...         News   
4      Pope Francis used his annual Christmas Day mes...         News   
...                                                  ...          ...   
23476  21st Century Wire says As 21WIRE reported earl...  Middle-east   
23477  21st Century Wire says It s a familiar theme. ...  Middle-east   
23478  Patrick Henningsen  21st Century WireRemember ...  Middle-east   
23479  21st Century Wire says Al Jazeera America will...  Middle-east   
23480  21st Century Wire says As 21WIRE predicted in ...  Middle-east   

                    date  
0      December 31, 2017  
1      December 31, 2017  
2      December 30, 2017  
3      December 29, 2017  
4      December 25, 2017  
...                  ...  
23476   January 16, 2016  
23477   January 16, 2016  
23478   January 15, 2016  
23479   January 14, 2016  
23480   January 12, 2016  

[23481 rows x 4 columns]

In [5]:
# Add column 'label'
df_true['label'] = 1
df_fake['label'] = 0

# Combine the DataFrame
df = pd.concat([df_true, df_fake], ignore_index=True)

In [6]:
df

title  \
0      As U.S. budget fight looms, Republicans flip t...   
1      U.S. military to accept transgender recruits o...   
2      Senior U.S. Republican senator: 'Let Mr. Muell...   
3      FBI Russia probe helped by Australian diplomat...   
4      Trump wants Postal Service to charge 'much mor...   
...                                                  ...   
44893  McPain: John McCain Furious That Iran Treated ...   
44894  JUSTICE? Yahoo Settles E-mail Privacy Class-ac...   
44895  Sunnistan: US and Allied ‘Safe Zone’ Plan to T...   
44896  How to Blow $700 Million: Al Jazeera America F...   
44897  10 U.S. Navy Sailors Held by Iranian Military ...   

                                                    text       subject  \
0      WASHINGTON (Reuters) - The head of a conservat...  politicsNews   
1      WASHINGTON (Reuters) - Transgender people will...  politicsNews   
2      WASHINGTON (Reuters) - The special counsel inv...  politicsNews   
3      WASHINGTON (Reuters) - Trump campaign adviser ...  politicsNews   
4      SEATTLE/WASHINGTON (Reuters) - President Donal...  politicsNews   
...                                                  ...           ...   
44893  21st Century Wire says As 21WIRE reported earl...   Middle-east   
44894  21st Century Wire says It s a familiar theme. ...   Middle-east   
44895  Patrick Henningsen  21st Century WireRemember ...   Middle-east   
44896  21st Century Wire says Al Jazeera America will...   Middle-east   
44897  21st Century Wire says As 21WIRE predicted in ...   Middle-east   

                     date  label  
0      December 31, 2017       1  
1      December 29, 2017       1  
2      December 31, 2017       1  
3      December 30, 2017       1  
4      December 29, 2017       1  
...                   ...    ...  
44893    January 16, 2016      0  
44894    January 16, 2016      0  
44895    January 15, 2016      0  
44896    January 14, 2016      0  
44897    January 12, 2016      0  

[44898 rows x 5 columns]

In [7]:
# Data cleaning
english_stopwords = stopwords.words('english')
nlp = spacy.load('en_core_web_sm')

def data_cleaner(sentence):
    sentence = sentence.lower()
    for c in string.punctuation:
        sentence = sentence.replace(c, " ")
    document = nlp(sentence)
    sentence = ' '.join(token.lemma_ for token in document)
    sentence = ' '.join(word for word in sentence.split() if word not in english_stopwords)
    sentence = re.sub('\d', '', sentence)
    return sentence


def clean_text_columns(df, text_columns):
    cleaned_data = []
    for index, row in df.iterrows():
        text = ''
        for column in text_columns:
            text += row[column] + ' '
        cleaned_data.append(data_cleaner(text))
    return cleaned_data


In [8]:
text_columns = ['title', 'text']
training_data_cleaned = clean_text_columns(df, text_columns)

# Apply the clean_text_columns function to clean the data
cleaned_texts = clean_text_columns(df, text_columns)

# Add the results to the DataFrame as a new column
df['cleaned_text'] = cleaned_texts

# Show the new column 'cleaned_text' in the DataFrame
print(df['cleaned_text'])


0        u budget fight loom republicans flip fiscal sc...
1        u military accept transgender recruit monday p...
2        senior u republican senator let mr mueller job...
3        fbi russia probe help australian diplomat tip ...
4        trump want postal service charge much amazon s...
                               ...                        
44893    mcpain john mccain furious iran treat sailor w...
44894    justice yahoo settle e mail privacy class acti...
44895    sunnistan ally ' safe zone ' plan take territo...
44896    blow  million al jazeera america finally call ...
44897     u navy sailor hold iranian military – sign ne...
Name: cleaned_text, Length: 44898, dtype: object


In [9]:
# Subdivision of the dataset
X_train, X_test, y_train, y_test = train_test_split(training_data_cleaned, df['label'], test_size=0.2, random_state=42)

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the counting vectorizer
vectorizer = CountVectorizer()

# Trasform textual data into a numeric rapresentation
X_train_text = vectorizer.fit_transform(X_train)
X_test_text = vectorizer.transform(X_test)


In [11]:



#  Creation of the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)
tfidf_train = tfidf_vectorizer.fit_transform(X_train)
tfidf_test = tfidf_vectorizer.transform(X_test)

# Creation of the Naive Bayes model
naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(tfidf_train, y_train)

# Model evaluation
predictions = naive_bayes_classifier.predict(tfidf_test)
print(classification_report(y_test, predictions))


              precision    recall  f1-score   support

           0       0.93      0.94      0.94      4650
           1       0.94      0.92      0.93      4330

    accuracy                           0.93      8980
   macro avg       0.93      0.93      0.93      8980
weighted avg       0.93      0.93      0.93      8980



<p> 
    These performance metrics indicate the effectiveness of a classification model. Here's what each metric means:

Precision: The precision for class 0 (0.93) means that out of all the instances predicted as class 0, 93% of them were actually class 0. Similarly, for class 1, it's 94%. In simpler terms, it measures the accuracy of positive predictions.

Recall (Sensitivity): The recall for class 0 (0.94) indicates that out of all the instances that were actually class 0, 94% of them were correctly predicted as class 0. For class 1, it's 92%. It measures the ability of the model to correctly identify all relevant instances.

F1-score: The F1-score is the harmonic mean of precision and recall. It provides a balance between precision and recall. For class 0, it's 0.94, and for class 1, it's 0.93.

Support: The number of actual occurrences of each class in the dataset. For class 0, it's 4650, and for class 1, it's 4330.

Accuracy: Overall accuracy of the model, which is the proportion of correctly classified instances among all instances. Here, it's 93%.

Macro avg: The unweighted mean of precision, recall, and F1-score across all classes. Here, it's 0.93.

Weighted avg: The weighted average of precision, recall, and F1-score, where each metric is weighted by the number of samples in each class. Here, it's also 0.93, as the dataset is balanced.

Overall, the model seems to perform well, with high precision, recall, and F1-score, indicating that it's good at both minimizing false positives and false negatives.





</p>

In [12]:
import matplotlib.pyplot as plt

def plot_fake_news_percentages(df):
    # # Calculate the percentages of fake news and the total news for each subject category
    categories = df['subject'].unique()
    fake_news_percentages = []
    total_news_counts = []

    for category in categories:
        category_df = df[df['subject'] == category]
        total_count = len(category_df)
        fake_count = (category_df['label'] == 0).sum()
        percentage_fake = (fake_count / total_count) * 100
        fake_news_percentages.append(percentage_fake)
        total_news_counts.append(total_count)

    
    categories_with_100_percent = [categories[i] for i in range(len(categories)) if fake_news_percentages[i] == 100]

    
    plt.figure(figsize=(10, 6))
    width = 0.35
    x = range(len(categories))

    plt.bar(x, total_news_counts, width, label='Total News', color='lightcoral')
    plt.ylabel('Total News Count', color='lightcoral')
    plt.tick_params(axis='y', labelcolor='lightcoral')

    plt.bar(x, fake_news_percentages, width, label='Fake News Percentage', color='skyblue')
    plt.ylabel('Fake News Percentage', color='skyblue')
    plt.tick_params(axis='y', labelcolor='skyblue')

    plt.title('Fake News Percentage and Total News Count per Subject Category')
    plt.xticks(x, categories, rotation=45, ha='right')

    
    for i in range(len(categories)):
        if categories[i] in categories_with_100_percent:
            plt.text(x[i], total_news_counts[i] + 1, '100%', ha='center')

    plt.legend(loc='upper right')
    plt.tight_layout()
    plt.grid()
    plt.show()


plot_fake_news_percentages(df_fake)



In [13]:
import pandas as pd
import gensim
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from gensim.parsing.preprocessing import preprocess_string



# Preprocessing of news headlines
def preprocess_titles(titles):
    processed_titles = [preprocess_string(title) for title in titles]
    return processed_titles

# Performing topic modeling on news headlines
fake_news_titles = df[df['label'] == 0]['title'].tolist()  
processed_titles = preprocess_titles(fake_news_titles)

dictionary = Dictionary(processed_titles)
corpus = [dictionary.doc2bow(title) for title in processed_titles]

#  LDA model training
num_topics = 5  # Numero di argomenti desiderati
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)

# Print dominant themes
print("Topic Modeling Results:")
for idx, topic in lda_model.print_topics(-1):
    print(f"Topic {idx}: {topic}")


Topic Modeling Results:
Topic 0: 0.014*"illeg" + 0.012*"video" + 0.011*"plan" + 0.010*"student" + 0.010*"democrat" + 0.007*"old" + 0.007*"alien" + 0.007*"school" + 0.007*"million" + 0.006*"year"
Topic 1: 0.046*"video" + 0.041*"trump" + 0.020*"hillari" + 0.013*"new" + 0.013*"watch" + 0.008*"support" + 0.008*"clinton" + 0.008*"presid" + 0.007*"cnn" + 0.007*"tweet"
Topic 2: 0.015*"video" + 0.009*"berni" + 0.009*"refuge" + 0.008*"state" + 0.008*"sander" + 0.008*"obama" + 0.007*"room" + 0.007*"muslim" + 0.007*"boiler" + 0.007*"fund"
Topic 3: 0.029*"video" + 0.029*"trump" + 0.013*"obama" + 0.011*"hillari" + 0.009*"clinton" + 0.008*"new" + 0.008*"presid" + 0.008*"break" + 0.007*"fbi" + 0.006*"wow"
Topic 4: 0.040*"video" + 0.020*"trump" + 0.013*"black" + 0.009*"protest" + 0.009*"watch" + 0.009*"anti" + 0.008*"room" + 0.008*"cop" + 0.008*"live" + 0.008*"polic"


<p> 
Topic 0: Topic 0: This topic appears to involve discussions related to various topics, including immigration ("illeg," "alien"), education ("student," "school"), and possibly economic issues ("million," "year"). The presence of terms like "video" and "plan" suggests that these discussions might be accompanied by or centered around video content or plans. It seems to be a diverse topic covering a range of social and policy-related issues.
    
Topic 1: This topic seems to be focused on videos related to Trump, with keywords like "video," "trump," "hillari," "watch," indicating discussions or content related to Trump and perhaps his opponent Hillary Clinton.

Topic 2: This topic appears to involve a mix of political figures like Bernie Sanders and Barack Obama, alongside terms related to refugees and fundraising, suggesting discussions related to political campaigns or humanitarian efforts.

Topic 3: Similar to Topic 1, this topic revolves around videos related to Trump and political figures like Obama and Clinton, with mentions of topics like the FBI, implying discussions about investigations or controversies.

Topic 4: This topic seems to involve videos related to protests, with keywords like "black," "protest," "anti," and terms related to law enforcement like "cop," "polic," indicating discussions or content related to social movements and police actions.</p>

In [14]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import nltk
nltk.download('punkt')


df_fake_news = pd.read_csv("Fake.csv")  

# Remove the stopwords
stop_words = set(stopwords.words('english'))


tokenized_titles = [word_tokenize(title.lower()) for title in df_fake_news['title']]


filtered_titles = [[word for word in title if word not in stop_words] for title in tokenized_titles]

# Calculation of keyword frequencies
all_words = [word for title in filtered_titles for word in title]
fdist = FreqDist(all_words)


print("10 parole più comuni nei titoli delle fake news:")
print(fdist.most_common(10))

# Topic Modeling with Latent Dirichlet Allocation (LDA)
vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
X = vectorizer.fit_transform(df_fake_news['title'])

#  LDA model training
num_topics = 5
lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda.fit(X)

# Show the 5 domanint themes
print("\n5 temi dominanti nei titoli delle fake news:")
for idx, topic in enumerate(lda.components_):
    print(f"Topic {idx + 1}:")
    feature_names = vectorizer.get_feature_names_out()
    top_words_idx = topic.argsort()[:-11:-1]  # Top 10 parole per ciascun topic
    top_words = [feature_names[i] for i in top_words_idx]
    print(top_words)



[nltk_data] Downloading package punkt to /Users/alessio/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


10 parole più comuni nei titoli delle fake news:
[('’', 13491), ('trump', 8694), (':', 8474), ('video', 8373), ('”', 5470), ('[', 5239), (']', 5238), ('“', 5028), (',', 4492), ('(', 4128)]

5 temi dominanti nei titoli delle fake news:
Topic 1:
['video', 'trump', 'obama', 'hillary', 'black', 'white', 'media', 'clinton', 'man', 'attack']
Topic 2:
['trump', 'video', 'watch', 'hillary', 'news', 'donald', 'just', 'clinton', 'fox', 'gop']
Topic 3:
['trump', 'video', 'just', 'obama', 'hillary', 'president', 'clinton', 'room', 'gets', 'tweets']
Topic 4:
['video', 'trump', 'obama', 'white', 'hillary', 'black', 'watch', 'bernie', 'house', 'sanders']
Topic 5:
['video', 'trump', 'obama', 'cruz', 'court', 'ted', 'muslim', 'breaking', 'new', 'gop']


<p>These topics seem to revolve around various political figures and current events. Here's a brief analysis of each:

Topic 1: This topic likely pertains to videos related to political figures such as Trump, Obama, and Hillary Clinton. Terms like "black," "white," and "media" suggest discussions on racial issues and media coverage.

Topic 2: This topic appears to focus on Trump-related videos, with mentions of Hillary Clinton, Fox News, and the GOP (Republican Party). It may involve discussions on recent news or events concerning Trump and the political landscape.

Topic 3: Similar to Topic 2, this topic revolves around Trump-related videos, along with Obama and Hillary Clinton. The mention of "president" and "tweets" indicates discussions on Trump's presidency and his Twitter activity.

Topic 4: This topic seems to involve videos related to Trump, Obama, Hillary Clinton, and Bernie Sanders. Terms like "white" and "black" suggest discussions on race, while "house" may refer to the White House or housing-related issues.

Topic 5: This topic likely discusses videos related to Trump, Obama, and Ted Cruz, with mentions of the court system, Muslim issues, and breaking news. It seems to cover a range of political and legal topics.

Overall, these topics reflect a mix of political figures, current events, and societal issues, demonstrating the breadth of discussions found in the analyzed corpus of videos. </p>

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans


X = df_fake_news['title'] + ' ' + df_fake_news['text']

# Data Vectorization
vectorizer = TfidfVectorizer()
X_vec = vectorizer.fit_transform(X)


num_clusters = 5  
kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init=10)
kmeans.fit(X_vec)

#  Show the 5 domanint themes
for i in range(num_clusters):
    print(f"Cluster {i}:")
    cluster_center = kmeans.cluster_centers_[i]
    top_features_idx = cluster_center.argsort()[:-11:-1]  # Top 10 feature per il cluster
    top_features = [vectorizer.get_feature_names_out()[idx] for idx in top_features_idx]
    print(top_features)



 


Cluster 0:
['boiler', 'acr', 'room', 'the', 'pm', 'join', 'radio', 'broadcast', '00', 'and']
Cluster 1:
['the', 'to', 'of', 'and', 'in', 'that', 'is', 'for', 'it', 'on']
Cluster 2:
['the', 'to', 'of', 'and', 'trump', 'in', 'video', 'is', 'on', 'that']
Cluster 3:
['the', 'clinton', 'to', 'hillary', 'of', 'and', 'her', 'she', 'in', 'that']
Cluster 4:
['the', 'trump', 'to', 'and', 'of', 'that', 'he', 'in', 'is', 'his']


In [16]:
# Loading a true news
df_true_news = pd.read_csv("True.csv")

# Use a title of true news
X_true = df_true_news['title']

# True news data vectorization
X_true_vec = vectorizer.transform(X_true)

# Applying clustering to true news
kmeans_genuine = KMeans(n_clusters=num_clusters, random_state=42, n_init=10)
kmeans_genuine.fit(X_true_vec)


for i in range(num_clusters):
    print(f"Cluster {i} (Genuine News):")
    cluster_center = kmeans_genuine.cluster_centers_[i]
    top_features_idx = cluster_center.argsort()[:-11:-1]  # Top 10 feature per il cluster
    top_features = [vectorizer.get_feature_names_out()[idx] for idx in top_features_idx]
    print(top_features)


Cluster 0 (Genuine News):
['house', 'senate', 'tax', 'white', 'bill', 'trump', 'to', 'panel', 'rico', 'healthcare']
Cluster 1 (Genuine News):
['to', 'says', 'in', 'myanmar', 'china', 'russia', 'of', 'for', 'on', 'turkey']
Cluster 2 (Genuine News):
['trump', 'says', 'to', 'on', 'in', 'russia', 'for', 'ban', 'travel', 'china']
Cluster 3 (Genuine News):
['eu', 'brexit', 'uk', 'britain', 'talks', 'says', 'deal', 'pm', 'may', 'to']
Cluster 4 (Genuine News):
['korea', 'north', 'china', 'south', 'says', 'sanctions', 'japan', 'to', 'trump', 'missile']


In [17]:
pip install textblob

Note: you may need to restart the kernel to use updated packages.


In [18]:

from textblob import TextBlob


# Function to perform sentiment analysis on a title
def analyze_sentiment(title):
    blob = TextBlob(title)
    sentiment = blob.sentiment.polarity
    return sentiment

# Apply sentiment analysis to all fake news title in the DataFrame
df_fake['sentiment'] = df_fake['title'].apply(analyze_sentiment)

# Show a title of the fake news with they sentiment
print(df_fake[['title', 'sentiment']])


                                                   title  sentiment
0       Donald Trump Sends Out Embarrassing New Year’...  -0.181818
1       Drunk Bragging Trump Staffer Started Russian ...  -0.250000
2       Sheriff David Clarke Becomes An Internet Joke...   0.000000
3       Trump Is So Obsessed He Even Has Obama’s Name...  -0.500000
4       Pope Francis Just Called Out Donald Trump Dur...   0.000000
...                                                  ...        ...
23476  McPain: John McCain Furious That Iran Treated ...   0.000000
23477  JUSTICE? Yahoo Settles E-mail Privacy Class-ac...   0.000000
23478  Sunnistan: US and Allied ‘Safe Zone’ Plan to T...   0.500000
23479  How to Blow $700 Million: Al Jazeera America F...   0.000000
23480  10 U.S. Navy Sailors Held by Iranian Military ...  -0.050000

[23481 rows x 2 columns]


<p> This kind of sentiment analysis can be useful for understanding public opinion or attitudes towards certain topics or personalities. Sentiment is represented by a numerical value, which could be positive, neutral, or negative, indicating the emotional evaluation of the headline. </p>

In [19]:
# Average sentiment polarity on fake news titles
average_sentiment_fake = df_fake['sentiment'].mean()

# Show the average sentiment polarity of fake news
print("Media della polarità del sentiment delle fake news:", average_sentiment_fake)

# Determine whether fake news titles tend to be positive, neutral or negative
if average_sentiment_fake > 0:
    print("I titoli delle fake news tendono ad essere positivi.")
elif average_sentiment_fake < 0:
    print("I titoli delle fake news tendono ad essere negativi.")
else:
    print("I titoli delle fake news hanno una polarità neutra.")


Media della polarità del sentiment delle fake news: 0.0032240714380715888
I titoli delle fake news tendono ad essere positivi.


In [20]:
import pickle

#Loading model from file 
with open('project.pkl', 'rb') as file:
    
    loaded_lda_model = pickle.load(file)




In [21]:
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

# Saving the TF-IDF vectorizer
with open('tfidf_vectorizer.pkl', 'wb') as file:
    pickle.dump(tfidf_vectorizer, file)

# Saving the Naive Bayes model
with open('naive_bayes_classifier.pkl', 'wb') as file:
    pickle.dump(naive_bayes_classifier, file)

# Loading the TF-IDF vectorizer
with open('tfidf_vectorizer.pkl', 'rb') as file:
    loaded_tfidf_vectorizer = pickle.load(file)

# Loading the Naive Bayes model 
with open('naive_bayes_classifier.pkl', 'rb') as file:
    loaded_naive_bayes_classifier = pickle.load(file)




In [22]:

fake_title_index = 19330  # Index of the line containing the title
fake_title = df_fake.loc[fake_title_index, "title"]  # using a title from fake_title_index
preprocessed_title = preprocess_string(fake_title)
tfidf_fake_title = loaded_tfidf_vectorizer.transform([fake_title])
predictions_fake_title = loaded_naive_bayes_classifier.predict(tfidf_fake_title)

print("Fake title:", fake_title)  # Stampa il titolo
print("Predicted label for the fake title:", predictions_fake_title[0])  

Fake title: REPORT: WHY HILLARY WONT’ ACCEPT Obama And Valerie Jarrett’s Invitation For Post-Inaugural Gathering To “Lay Groundwork For The Offensive Against Trump”
Predicted label for the fake title: 0
